In [99]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
from urllib.parse import urlparse

In [100]:
url_base='https://www.ylh.gov.tw'
url_work='/?aid=104'
url_full=url_base+url_work
g=requests.get(url_full)
soup=BeautifulSoup(g.content, 'html.parser')

# Get Pages link

In [106]:
pages_link=[]
for link in soup.find_all('li',class_="page-item"):
    if link.find("a"):
        if link.find("a") == -1 or link.text == "第一頁" or link.text == "«" or link.text == "»" or link.text == "最末頁":
            pass
        elif link.find('a').has_attr("disabled"):
            print('skip:',link.text)
            pages_link.append(url_full)
        else:
            pages_link.append(url_base+link.find('a').get('href'))
pages_link

skip: 3


['https://www.ylh.gov.tw/?aid=104&pid=0&page_name=list&pageNo=1',
 'https://www.ylh.gov.tw/?aid=104&pid=0&page_name=list&pageNo=2',
 'https://www.ylh.gov.tw/?aid=104',
 'https://www.ylh.gov.tw/?aid=104&pid=0&page_name=list&pageNo=4']

# Get work table

In [8]:
soup.find_all('table')[0].find('tbody')

<tbody>
<tr>
<td class="align-middle" data-title="標題" width="50%">公告本院護理部公開甄選院聘護理師(手術室)儲備20名(隨到隨招，請於上班時間先行電話預約)</td>
<td class="text-center align-middle" data-title="公告日期">2022-01-18</td>
<td class="text-center align-middle" data-title="截止日期">2022-07-31</td>
<td class="text-center align-middle" data-title="下載">
<a href="/public/jobnews/4eb1cf3e5ea8a5858cb8f631988ab5a8.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="[PDF]下載  (另開新視窗) ">查看</a>
</td>
</tr>
<tr>
<td class="align-middle" data-title="標題" width="50%">公告本院護理部公開甄選院聘護理師/實習護士儲備30名(隨到隨招，請於上班時間先行電話預約)</td>
<td class="text-center align-middle" data-title="公告日期">2021-12-23</td>
<td class="text-center align-middle" data-title="截止日期">2022-07-31</td>
<td class="text-center align-middle" data-title="下載">
<a href="/public/jobnews/ba6bb249a04a65caa2026732b239f9bb.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="[PDF]下載  (另開新視窗) ">查看</a>
</td>
</tr>
<tr>
<td class="align-middle" data-title

In [102]:
def get_each_page_wrok_table(soup,work_table):
    for work in soup.find_all('table',class_="table table-striped table-hover news_table")[0].find("tbody").find_all("tr"):
        all_td = work.find_all("td")
        title = all_td[0].text
        place = '察看連結的簡章'
        deadline = all_td[2].text
        
        link=url_base+all_td[3].find('a').get('href')
        if "錄取名單" in title:
            print('skip: ',title)
        elif "甄試名單" in title:
            print('skip: ',title)
        elif "複試名單" in title:
            print('skip: ',title)
        elif "志願服務工作" in title:
            print('skip: ',title)
        elif "甄審會" in title:
            print('skip: ',title)
        elif "時薪人員" in title:
            print('skip: ',title)
        else:
            originization = re.search("\B院((.*)[室,部,中心])",title).group(1)
            new_title = re.search("\B聘((.*)[員,師,廚])",title).group(1)
            print("召聘職稱",new_title,'院區',place,"截止日期",deadline,"職缺單位",originization,"links",link)
            work_table.append([new_title, deadline, originization, place, link ])
    return work_table

In [103]:
def get_base_web_data(url_base,url_work_table=None):
    if url_work_table != None:
        url = url_base + url_work_table
    else:
        url = url_base
    g=requests.get(url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [104]:
work_table = []
for page in pages_link:
    soup = get_base_web_data(page)
    work_table = get_each_page_wrok_table(soup,work_table)

召聘職稱 護理師 院區 察看連結的簡章 截止日期 2022-07-31 職缺單位 護理部公開甄選院聘護理師(手術室 links https://www.ylh.gov.tw/public/jobnews/4eb1cf3e5ea8a5858cb8f631988ab5a8.pdf
召聘職稱 護理師 院區 察看連結的簡章 截止日期 2022-07-31 職缺單位 護理部 links https://www.ylh.gov.tw/public/jobnews/ba6bb249a04a65caa2026732b239f9bb.pdf
skip:  公告本院婦產部公開甄選院聘助理管理師1名【甄試名單】
skip:  公告本院腫瘤醫學部111年4月27日公開甄選院聘醫事放射師1名【錄取名單】
skip:  公告本院神經部111年4月21日公開甄選院聘醫事檢驗師1名【錄取名單】
skip:  公告本院企劃管理室111年4月28日公開甄選院聘管理師職務代理人1名「錄取名單」
召聘職稱 契約主治醫師 院區 察看連結的簡章 截止日期 2022-05-20 職缺單位 環境及職業醫學部 links https://www.ylh.gov.tw/public/jobnews/21d8787d13c98501fcae8cce12b762cd.pdf
skip:  公告本院外科部(傷造口中心)111年4月19日公開甄選院聘(傷造口)資深/專科護理師1名《錄取名單》
召聘職稱 助理管理師 院區 察看連結的簡章 截止日期 2022-05-09 職缺單位 主計室公開甄選院聘助理管理師(身心 links https://www.ylh.gov.tw/public/jobnews/e994b1886e7bfdaf6f4ae019a3a24aee.pdf
召聘職稱 副管理師 院區 察看連結的簡章 截止日期 2022-05-10 職缺單位 護理部 links https://www.ylh.gov.tw/public/jobnews/335f84d1c6cca748ca2b5d6d190feb22.pdf
召聘職稱 醫事檢驗師 院區 察看連結的簡章 截止日期 2022-05-20 職缺單位 病理部 links https://www.ylh.gov.tw/public/jobnews/94446d391604b

In [75]:
re.search("\B院((.*)部)","公告本院影像醫學部111年4月1日公開甄選院聘護理師1名錄取名單").group(1)

'影像醫學部'

In [87]:
re.search("\B聘((.*)師)","公告本院護理部公開甄選院聘護理師").group(1)

'護理師'

In [105]:
work_table

[['護理師',
  '2022-07-31',
  '護理部公開甄選院聘護理師(手術室',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/4eb1cf3e5ea8a5858cb8f631988ab5a8.pdf'],
 ['護理師',
  '2022-07-31',
  '護理部',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/ba6bb249a04a65caa2026732b239f9bb.pdf'],
 ['契約主治醫師',
  '2022-05-20',
  '環境及職業醫學部',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/21d8787d13c98501fcae8cce12b762cd.pdf'],
 ['助理管理師',
  '2022-05-09',
  '主計室公開甄選院聘助理管理師(身心',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/e994b1886e7bfdaf6f4ae019a3a24aee.pdf'],
 ['副管理師',
  '2022-05-10',
  '護理部',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/335f84d1c6cca748ca2b5d6d190feb22.pdf'],
 ['醫事檢驗師',
  '2022-05-20',
  '病理部',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/94446d391604bd99f97a95fe3344c187.pdf'],
 ['佐理員',
  '2022-05-09',
  '護理部(供應室',
  '察看連結的簡章',
  'https://www.ylh.gov.tw/public/jobnews/45ef90046e200f641a8e9cb97c23b2da.pdf'],
 ['助廚',
  '2022-12-31',
  '營養室',
  '察看連結的簡章',
  'https://www